## Uniswap v3 Historical Fee Simulation — High-Accuracy Plan

Objective: Build a tick-precise, contract-faithful simulator to compute fees for arbitrary ranges and times. This mirrors Uniswap v3 core: per-tick swap traversal, exact fee growth accounting, protocol fee handling, and position fee queries using fee growth inside.

### 0) Fidelity goals
- Replicate core mechanics:
  - Tick-by-tick swap progression (`swapStep`), exact rounding rules.
  - Active liquidity updates at tick crossings via `liquidityNet`.
  - Global fee growth in Q128 per liquidity (token0/1), and `feeGrowthOutside` updates at tick crossings.
  - Protocol fee extraction per token (from `slot0.feeProtocol`).
- Deterministic integer math only (Q64.96 for prices, Q128 for fee growth), no floating-point.
- Support queries for any `(tickLower, tickUpper)` over a window via `feeGrowthInside` deltas (fast for RL search).

### 1) Data requirements and initialization
- **Mandatory**
  - Swaps: `amount0, amount1, sqrtPriceX96, tick, liquidity, evt_block_time` in chronological order.
  - Mints/Burns: full history up to and through the simulation window, or a tick snapshot at `t0`.
  - Pool config: `fee`, `tickSpacing`, `token0/1`, token decimals.
  - Slot0 snapshots: for initial state (`sqrtPriceX96`, `tick`, `feeProtocol`).
- **Tick state at t0 (critical)**
  - Build `liquidityNet` map at tick boundaries as of `t0`:
    - Option A (preferred): load a tick snapshot (liquidityGross/liquidityNet per initialized tick).
    - Option B: reconstruct by replaying all `mints`/`burns` from pool inception to `t0`.
  - Initialize active liquidity `L_active` from `slot0` at `t0`:
    - Sum `liquidityNet` for ticks ≤ current `tick`, respecting v3 core convention.

### 2) Core state and units
- Global state (at any time):
  - `sqrtPriceX96` (Q64.96), `tick` (int), `L_active` (uint128), `fee` (uint24, hundredths of a bip).
  - `feeProtocol` (two 4-bit values packed in `slot0.feeProtocol` → token0, token1 share).
  - `feeGrowthGlobal0X128`, `feeGrowthGlobal1X128` (uint256, Q128 per liquidity unit).
- Per tick boundary `t`:
  - `liquidityNet[t]` (int128): net liquidity change when crossing `t` upward.
  - `feeGrowthOutside0X128[t]`, `feeGrowthOutside1X128[t]`.
- Position query uses only growth deltas; no need to track per-position state during simulation.

### 3) Exact math (from whitepaper / core)
- Price and ticks:
  - `P = sqrtPriceX96 / 2^96`; token1/token0 price = `P^2 * 10^(dec0 - dec1)` (for display only).
  - `tick <-> sqrtPrice`: `sqrtPriceAtTick(t)`, `tickAtSqrtPrice(P)` per v3 formulas.
- Liquidity math (amounts vs L):
  - Use standard v3 `getAmount0Delta`, `getAmount1Delta` with exact rounding rules.
- Fee accounting per step:
  - Given step input amount and fee, `feeAmount = floor(amountIn * fee / 1e6)`.
  - Protocol fee share per token: extracted via `feeProtocol`; subtract from `feeAmount` before adding to growth.
  - `feeGrowthGlobal{0,1}X128 += (feeAmount_remaining << 128) / L_active`.
- Tick crossing:
  - When crossing tick `t` upward:
    - `feeGrowthOutside{0,1}[t]` updated to current `feeGrowthGlobal{0,1}X128` if first time on that side.
    - `L_active += liquidityNet[t]`.
  - Crossing downward applies inverse net and outside updates (mirror logic).

### 4) Swap simulation loop (contract-faithful)
For each swap event (final `sqrtPriceX96_f`, `tick_f`, and signed `amount0/1`):
1. Starting state: previous event's final state (or `slot0` at `t0`).
2. Determine direction and input token from sign of `amount{0,1}` (positive indicates input for that token in Uniswap v3 logs).
3. While current `sqrtPriceX96` != `sqrtPriceX96_f`:
   - Compute next tick boundary `sqrtPriceNext = sqrtPriceAtTick(nextTick)` based on direction.
   - Run `swapStep`:
     - Compute amount to move to `sqrtPriceNext` or to final, considering fee-on-input and rounding rules.
     - Compute `feeAmount` for input token, split protocol vs LPs.
     - Update `feeGrowthGlobal{input}` by `(feeAmount_LP << 128) / L_active`.
     - Update `sqrtPriceX96`, `tick`, and if boundary reached, apply `feeGrowthOutside` updates and `L_active += liquidityNet`.
4. Validate that end state matches event (`sqrtPriceX96_f`, `tick_f`, `liquidity` if provided). Log any deltas for diagnostics.

### 5) Position fee queries (any range, any time window)
- For window `[t0, t1]`, capture snapshots of:
  - `feeGrowthGlobal{0,1}X128(t0)`, `(t1)` and all `feeGrowthOutside{0,1}X128` at both times.
- For a position `(tickLower, tickUpper)`:
  - `feeGrowthInside = feeGrowthGlobal - feeGrowthBelow - feeGrowthAbove` per v3:
    - Below/Above computed from `feeGrowthOutside` using current tick at each snapshot.
  - `fees_token0 = L_pos * (feeGrowthInside0X128(t1) - feeGrowthInside0X128(t0)) >> 128` (likewise token1).
- This enables fast evaluation for many candidate ranges (ideal for RL).

### 6) Protocol fee handling
- Decode `slot0.feeProtocol` per token at each time (two 4-bit values packed):
  - Implement exact core semantics: LP share = `feeAmount * (256 - protocolFee) / 256` (or per-core rule).
  - If the dataset lacks precise protocol fee changes, default to 0 and allow override.

### 7) Robustness, ordering, and edge cases
- Event ordering: strictly sort by `(block_number, log_index)` if available; otherwise timestamp, then tx hash.
- Zero or tiny liquidity: skip steps if `L_active == 0` (no fee growth possible); warn.
- Tick extremes: guard against min/max tick; clamp as in core.
- Duplicate timestamps: process sequentially; state continuity is defined by prior event.
- Gaps: price changes only via swaps; `slot0` is used only for initial state validation.

### 8) Performance plan (for RL workloads)
- Precompute growth timelines:
  - Maintain cumulative `feeGrowthGlobal{0,1}X128` and `feeGrowthOutside{0,1}X128` over the window.
  - Optional: persist per-tick cumulative arrays to disk for fast random access.
- Query engine:
  - Given many `(tickLower, tickUpper)` and `(t0, t1)`, compute fees with O(1)/O(log N) using growth snapshots.
- Parallelization: shard by time or by candidate ranges.

### 9) Validation
- Fee mass balance: sum of LP fees equals total fee paid minus protocol fees.
- End-state consistency: simulated `sqrtPriceX96`, `tick`, `L_active` equal event finals.
- Cross-check with spot The Graph pools or known backtests for sample windows.

### 10) Implementation checklist
- Parsers: swaps, mints, burns, slot0, pool config, tokens.
- Tick map builder: from snapshot or full event replay to `t0`.
- Core math primitives: `sqrtPriceAtTick`, `tickAtSqrtPrice`, `getAmount{0,1}Delta` with rounding.
- Swap engine with `swapStep` loop and tick crossing.
- Growth stores: globals and per-tick outside.
- Position fee query: `feeGrowthInside` delta → token0/1 fees.
- Test suite: unit tests on math, step transitions, and an end-to-end small window.

If you approve this plan, I’ll implement the stateful engine and growth accounting first, then add the position query and validation utilities.
